In [35]:
import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_PC.csv", low_memory=False)

#read dedupe file
df = pd.read_csv("outputs/probable_duplicates_CSD.csv")

def hierarchy(id1, id2): 

    try:

        check_beds1 = final.loc[final.idx == id1].number_beds.item()
        check_beds2 = final.loc[final.idx == id2].number_beds.item()
        check_type1 = final.loc[final.idx == id1].facility_type.item()
        check_type2 = final.loc[final.idx == id2].facility_type.item()
        check_file1 = final.loc[final.idx == id1].filename.item()
        check_file2 = final.loc[final.idx == id2].filename.item()
        check_name1 = final.loc[final.idx == id1].facility_name.item()
        check_name2 = final.loc[final.idx == id2].facility_name.item()

        filename_priority = ['QC_healthcare_facilities.csv', 'QC_Gatineau_public_places_including_hospitals,csv', 'QC_Quebec_City_public_places.csv', 
                             'ON_Ottawa_hospital.csv', 'ON_Durham_healthcare_facilities.csv', 'ON_healthcare_facilities.csv', 'NB_hospital_horizon.csv', 'NB_nursing_homes.csv']

        facilityname_priority = ['hospital', 'hôpita','hopita']

        if filename_priority.count(check_file1) == 1:

            if filename_priority.count(check_file2) == 1:

                priority1 = filename_priority.index(check_file1)
                priority2 = filename_priority.index(check_file2)

                if priority1 > priority2:
                    return id1
                elif priority1 < priority2:
                    return id2
                elif priority1 == priority2:

                    if any(substring in check_name1.lower() for substring in facilityname_priority):
                        return id2
                    elif any(substring in check_name2.lower() for substring in facilityname_priority):
                        return id1
                    else:
                        return id2

            elif filename_priority.count(check_file2) == 0:
                return id2

        elif filename_priority.count(check_file1) == 0:

            if filename_priority.count(check_file2) == 1:
                return id1

            elif filename_priority.count(check_file2) == 0:

                if check_file1 == 'CIHI_healthcare_facilities.csv' and check_file2 != 'CIHI_healthcare_facilities.csv':
                    return id1

                elif check_file2 == 'CIHI_healthcare_facilities.csv' and check_file1 != 'CIHI_healthcare_facilities.csv':
                    return id2

                else:

                    if check_beds1 == np.nan and check_beds2 == np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2

                    elif check_beds1 == np.nan and check_beds2 != np.nan:
                        return id1

                    elif check_beds1 != np.nan and check_beds2 == np.nan:
                        return id2

                    elif check_beds1 != np.nan and check_beds2 != np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2



    except ValueError:
        return "Item not found."


df["to_remove"] = df.apply(lambda x: hierarchy( id1 = x.idx1, id2 = x.idx2), axis = 1)
#df.to_csv('test.csv')


In [36]:
def check_1(address, distance, name, streetnumber, types):

    if distance != None:
        if address >= 0.75 and distance < 2 and name > 0.7 and streetnumber == 1 and types == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_1"] = df.apply(lambda x: check_1(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match,
                          types = x.Type_Match), axis=1)

#df.to_csv('test.csv')

In [37]:
def check_2(address, distance, name, types):

    if distance != None:
        if address >= 0.75 and distance < 1 and name > 0.85 and types == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_2"] = df.apply(lambda x: check_2(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          types = x.Type_Match), axis=1)


In [38]:
def check_3(address, distance, file, types):

    if distance != None:
        if address >= 0.99 and distance <= 0.5 and file == 1 and types == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_3"] = df.apply(lambda x: check_3(address = x.Addr_CS, 
                          distance = x.Distance,
                          file = x.File_Match,
                          types = x.Type_Match), axis=1)

In [39]:
def check_4(address, distance, streetnumber, types):


    if distance != None:
        if address >= 0.8 and distance < 0.1 and streetnumber == 1 and types == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_4"] = df.apply(lambda x: check_4(address = x.Addr_CS, 
                          distance = x.Distance, 
                          streetnumber = x.StrNum_Match,
                          types = x.Type_Match), axis=1)

In [40]:
def check_5(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.90 and distance < 2 and name > 0.93 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_5"] = df.apply(lambda x: check_5(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

In [41]:
# remove duplicates

rows = df.index[(df['Check_1'] == True) | (df['Check_2'] == True) | (df['Check_3'] == True) | (df['Check_4'] == True) | (df['Check_5'] == True)].tolist()
false_rows = df.index[(df['Check_1'] == False) & (df['Check_2'] == False) & (df['Check_3'] == False) & (df['Check_4'] == False) & (df['Check_5'] == False)].tolist()
print(rows)

df_true = df.loc[rows]
df_false = df.loc[false_rows]

df_true.to_csv('outputs/duplicates_CSD.csv', index=False)
df_false.to_csv('outputs/NOT_duplicates_CSD.csv', index=False)
print(df_true)

hashes = df_true['to_remove'].tolist()
print(hashes)

final = final[~final['idx'].isin(hashes)]
final.to_csv('deduplicated_CSD.csv', index=False)
final.to_csv('inputs/deduplicated_CSD.csv', index=False)



[49, 50, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 65, 66, 67, 68, 69, 87, 89, 122, 123, 131, 136, 139, 145, 154, 156, 157, 158, 159, 160, 161, 166, 171, 173, 175, 178, 180, 187, 189, 192, 194, 196, 200, 202, 203, 204, 206, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 243, 244, 246, 247, 248, 249, 250, 251, 252, 253, 255, 256, 257, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 277, 278, 279, 280, 281, 282, 286, 288, 289, 290, 291, 292, 293, 294, 315, 319, 323, 327, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 341, 347, 373, 403, 404, 413, 469, 470, 489, 496, 528, 529, 532, 542, 544, 546, 550, 551, 552, 553, 554, 556, 557, 559, 560, 562, 563, 593, 622, 623, 630, 631, 635, 637, 640, 651, 659, 671, 672, 687, 709, 713, 714, 718, 724, 729, 734, 745, 753, 759, 762, 763, 766, 782, 791, 792, 813, 817, 819, 820, 828, 843, 845, 849, 851